In [4]:
# Magic comands
%reload_ext autoreload
%autoreload 2

# Libraries
import os, sys
import numpy as np

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

origin_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(origin_path, os.pardir))
os.chdir(parent_dir)

from app.nqueens import NQueensGeneticAlgorithm

In [5]:
def genetic_algorithm(
                    n = 8,
                    pop = 200,
                    gen = 200,
                    mut_perc = .05,
                    cross_perc = 0.75,
                    mut='swap_mutation',
                    cross='pmx',
                    select='tournament_selection',
                    ):
    
    nQueensGA = NQueensGeneticAlgorithm(population_size=pop, dimension=n)

    nQueensGA.run(
        generations=gen,
        mutation_probability=mut_perc,
        crossover_probability=cross_perc,
        mutation_operator=mut,
        crossover_operator=cross,
        selection_operator=select
    )

    return nQueensGA.report()

In [6]:
# testing function
ga = genetic_algorithm(
                                            n = 8,
                                            pop = 100,
                                            gen = 500,
                                            mut_perc = .02,
                                            cross_perc = .8,
                                            mut = "inversion_mutation",
                                            cross = "single_cross",
                                            select = "tournament_selection"
                                            )
ga

{'generations': 500,
 'duration': 197,
 'best_fitness': 27,
 'best_fitness_percentage': 96.42857142857143,
 'best_representation': [7, 4, 2, 0, 6, 1, 7, 5],
 'worst_fitness': 27,
 'worst_representation': [7, 4, 2, 0, 6, 1, 7, 5],
 'mean_fitness': 27.0,
 'selection_operator': 'tournament_selection',
 'mutate_operator': 'inversion_mutation',
 'crossover_operator': 'single_point_co'}

## __1. Optimization__

In [7]:

def best_operators_search(parameter: str, values: list):
    # dictionary with parameter names and default values 
    defaults = genetic_algorithm.__defaults__
    parameters = genetic_algorithm.__code__.co_varnames
    params = {param: val for param, val in zip(parameters, defaults)}
    # initializing variables
    best_fit = 0.0
    time_fit = 9e99
    fit_dict = {}
    time_dict = {}
    param_list = []

    # Perform grid search
    for i in values:
        # 100 iterations to generate statistically relevant results
        iter = 1
        fit_list = []
        time_list = []
        for _ in range(100):
            # changing parameter according to input
            params[parameter] = i
            # Define the model with current hyperparameters
            ga = genetic_algorithm(**params)
            # gets restults
            fit = ga['best_fitness_percentage']
            time_it = ga['duration']
            # Check if current solution has better fitness than previous best 
            if fit > best_fit:
                best_fit = fit
                time_fit = time_it
                best_param = {parameter: i}
            # Check if the solution with fitness as good is faster than previous best
            if fit == best_fit and time_it < time_fit:
                time_fit = time_it
                best_param = {parameter: i}
            # Append results to get 100 iterations values
            fit_list.append(best_fit)
            time_list.append(time_fit)
        # saving results per parameter
        param_list.append(best_param)
        fit_dict[i] = fit_list
        time_dict[i] = time_list
        
    return best_fit, time_fit, fit_dict, time_dict, param_list

### __1.1 Selection Module__

In [8]:
# Define the range of values for each hyperparameter
#xo = ['single_cross', 'cycle_cross', 'pmx', 'arithmetic_cross']
select = ['tournament_selection', 'stochastic_universal_sampling']
#mutation = ['swap_mutation', 'random_mutation', 'inversion_mutation']
#population = [100, 250, 500, 1000]
#generations = [100, 250, 500]
#cross_percentage = np.arange(0.8, 0.95, 0.05)
#mut_percentage = np.arange(0.01, 0.06, 0.01)

best_operators_search('select', select)